In [246]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
from pprint import pprint
import os
import matplotlib.pyplot as plt
import pickle

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Scraping from allmenus.com

In [45]:
url = 'https://www.allmenus.com/ny/new-york/368140-somtum-der/menu/'
r = requests.get(url)
page = r.text
soup = BeautifulSoup(page)

/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [46]:
somtumder_menu = {}

for item in soup.find_all(class_='menu-items'):
    somtumder_menu[item.find(class_='item-title').text] = (item.find(class_='description').text,
                                                           item.find(class_='item-price').text.strip())
    

# Scraping from menupages.com

In [8]:
url3 = 'https://menupages.com/somtum-der/85-ave-a-new-york'
r3 = requests.get(url3)
page3 = r3.text
soup3 = BeautifulSoup(page3, 'html.parser')

In [71]:
somtumder_menu_2 = {}

for item in soup3.find_all(class_='menu-item'):
    dish = item.find(class_='menu-item__title').text
    if item.find(class_='menu-item__description') is None:
        desc = ''
    else:
        desc = item.find(class_='menu-item__description').text.strip()
    price = item.find(class_='menu-item__price').text.strip()
    somtumder_menu_2[dish] = (desc, price)

In [43]:
soup3.find_all(class_='restaurant-phone')[0].find('a')['href'].replace('tel:','')

'+12122608570'

In [44]:
def get_site_pages(page_list, n_pages):
    for i in range(2, n_pages+1):
        page_list.append('https://menupages.com/restaurants/ny-new-york/'+str(i))
    return page_list


def get_menu_urls(page_list):
    menu_urls = []
    for page in page_list:
        r = requests.get(page)
        txt = r.text
        soup = BeautifulSoup(txt, 'html.parser')
        for item in soup.find_all(class_='restaurant__title'):
            menu_urls.append('https://menupages.com'+item.find('a')['href'])
    return menu_urls

In [45]:
page_list = get_site_pages(['https://menupages.com/restaurants/ny-new-york'], 86)
menu_urls = get_menu_urls(page_list)

In [51]:
def get_phone(menu_urls):
    phone_book = {}
    
    for menu in menu_urls:
        r = requests.get(menu)
        txt = r.text
        soup = BeautifulSoup(txt, 'html.parser')
        
        name = soup.find_all(class_='header__restaurant-name')
        if len(name) == 0:
            continue
        
        phone_book['name'] = name[0].text.strip()
        phone_book['phone'] = soup.find_all(class_='restaurant-phone')[0].find('a')['href'].replace('tel:','')
    
        print(phone_book['name'])
        print(phone_book['phone'])
    return phone_book

# Getting Yelp data from API using phone number

In [4]:
with open('data/menus_new.json') as f:
    data = json.load(f)

In [7]:
restaurant_list = [rest['name'] for rest in data]
phone_list = [rest['phone'] for rest in data]

In [77]:
headers = {'Authorization': 'bearer %s' % os.environ['YELP_API_KEY']}
biz_data = []

for i, phone in enumerate(phone_list):
    print(i)
    params = {'phone': phone}
    r = requests.get('https://api.yelp.com/v3/businesses/search/phone',
                     params=params, headers=headers)
    biz_data.append(r.json())

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699


# Cleaning Yelp API data

In [203]:
cleaned_biz_data = biz_data.copy()

In [204]:
def remove_all(lst, val):
    """Remove all instances of specified value from a list"""
    lst[:] = (x for x in lst if x != val)

In [205]:
for biz in cleaned_biz_data:
    try: 
        for listing in biz['businesses']:
            if listing['location']['state'] != 'NY':
                remove_all(biz['businesses'], listing)
        if len(biz['businesses']) == 0:
            remove_all(cleaned_biz_data, biz)
    except KeyError:
        remove_all(cleaned_biz_data, biz)

# Getting relevant URLs from cleaned Yelp data

In [211]:
yelp_urls = []
for biz in cleaned_biz_data:
    yelp_urls.append( [biz['businesses'][i]['url'] for i in range(len(biz['businesses'])) ] )

3805


In [213]:
flat_urls = [url for restaurant in yelp_urls for url in restaurant]

In [223]:
print(len(flat_urls))
print((len(set(flat_urls)) ))
print(len(yelp_urls))

4915
4551
3805


In [240]:
review_urls = list(set(flat_urls))

In [241]:
sep = '?adjust_creative'
clean_urls = [r.split(sep)[0] for r in review_urls]

In [258]:
len(clean_urls)

4551

In [259]:
yelp_reviews = []

for i, t in enumerate(clean_urls):
    print(i, ' of ', len(clean_urls))
    try:
        r = requests.get(t)
        soup = BeautifulSoup(r.text, 'html.parser')

        phone = soup.find(class_='biz-phone').text.strip().replace('(','+').replace(')','').replace('-','').replace(' ','')

        reviews = []
        for item in soup.find_all(class_='review-content'):
            for p in item.find_all('p',{'lang':'en'}):
                for br in p.find_all('br'):
                    br.replace_with('\n')
                reviews.append(p.text)

        rest_dict = {'phone': phone}
        rest_dict['reviews'] = reviews

        yelp_reviews.append(rest_dict)
    
    except AttributeError:
        print('error at index ',i)
        continue

0  of  4551
1  of  4551
2  of  4551
3  of  4551
4  of  4551
5  of  4551
6  of  4551
7  of  4551
8  of  4551
9  of  4551
10  of  4551
11  of  4551
12  of  4551
13  of  4551
14  of  4551
15  of  4551
16  of  4551
17  of  4551
18  of  4551
19  of  4551
20  of  4551
21  of  4551
22  of  4551
23  of  4551
24  of  4551
25  of  4551
26  of  4551
27  of  4551
28  of  4551
29  of  4551
30  of  4551
31  of  4551
32  of  4551
33  of  4551
34  of  4551
35  of  4551
36  of  4551
37  of  4551
38  of  4551
39  of  4551
40  of  4551
41  of  4551
42  of  4551
43  of  4551
44  of  4551
45  of  4551
46  of  4551
47  of  4551
48  of  4551
49  of  4551
50  of  4551
51  of  4551
52  of  4551
53  of  4551
54  of  4551
55  of  4551
56  of  4551
57  of  4551
58  of  4551
59  of  4551
60  of  4551
61  of  4551
62  of  4551
63  of  4551
64  of  4551
65  of  4551
66  of  4551
67  of  4551
68  of  4551
69  of  4551
70  of  4551
71  of  4551
72  of  4551
73  of  4551
74  of  4551
75  of  4551
76  of  4551
77  of  4

592  of  4551
593  of  4551
594  of  4551
595  of  4551
596  of  4551
597  of  4551
598  of  4551
599  of  4551
600  of  4551
601  of  4551
602  of  4551
603  of  4551
604  of  4551
605  of  4551
606  of  4551
607  of  4551
608  of  4551
609  of  4551
610  of  4551
611  of  4551
612  of  4551
613  of  4551
614  of  4551
615  of  4551
616  of  4551
617  of  4551
618  of  4551
619  of  4551
620  of  4551
621  of  4551
622  of  4551
623  of  4551
624  of  4551
625  of  4551
626  of  4551
627  of  4551
628  of  4551
629  of  4551
630  of  4551
631  of  4551
632  of  4551
633  of  4551
634  of  4551
635  of  4551
636  of  4551
637  of  4551
638  of  4551
639  of  4551
640  of  4551
641  of  4551
642  of  4551
643  of  4551
644  of  4551
645  of  4551
646  of  4551
647  of  4551
648  of  4551
649  of  4551
650  of  4551
651  of  4551
652  of  4551
653  of  4551
654  of  4551
655  of  4551
656  of  4551
657  of  4551
658  of  4551
659  of  4551
660  of  4551
661  of  4551
662  of  4551
663  o

1166  of  4551
1167  of  4551
1168  of  4551
1169  of  4551
1170  of  4551
1171  of  4551
1172  of  4551
1173  of  4551
1174  of  4551
1175  of  4551
1176  of  4551
1177  of  4551
1178  of  4551
1179  of  4551
1180  of  4551
1181  of  4551
1182  of  4551
1183  of  4551
1184  of  4551
1185  of  4551
1186  of  4551
1187  of  4551
1188  of  4551
1189  of  4551
1190  of  4551
1191  of  4551
1192  of  4551
1193  of  4551
1194  of  4551
1195  of  4551
1196  of  4551
1197  of  4551
1198  of  4551
1199  of  4551
1200  of  4551
1201  of  4551
1202  of  4551
1203  of  4551
1204  of  4551
1205  of  4551
1206  of  4551
1207  of  4551
1208  of  4551
1209  of  4551
1210  of  4551
1211  of  4551
1212  of  4551
1213  of  4551
1214  of  4551
1215  of  4551
1216  of  4551
1217  of  4551
1218  of  4551
1219  of  4551
1220  of  4551
1221  of  4551
1222  of  4551
1223  of  4551
1224  of  4551
1225  of  4551
1226  of  4551
1227  of  4551
1228  of  4551
1229  of  4551
1230  of  4551
1231  of  4551
1232  of  

1711  of  4551
1712  of  4551
1713  of  4551
1714  of  4551
1715  of  4551
1716  of  4551
1717  of  4551
1718  of  4551
1719  of  4551
1720  of  4551
1721  of  4551
1722  of  4551
1723  of  4551
1724  of  4551
1725  of  4551
1726  of  4551
1727  of  4551
1728  of  4551
1729  of  4551
1730  of  4551
1731  of  4551
1732  of  4551
1733  of  4551
1734  of  4551
1735  of  4551
1736  of  4551
1737  of  4551
1738  of  4551
1739  of  4551
1740  of  4551
1741  of  4551
1742  of  4551
1743  of  4551
1744  of  4551
1745  of  4551
1746  of  4551
1747  of  4551
1748  of  4551
1749  of  4551
1750  of  4551
1751  of  4551
1752  of  4551
1753  of  4551
1754  of  4551
1755  of  4551
1756  of  4551
1757  of  4551
1758  of  4551
1759  of  4551
1760  of  4551
1761  of  4551
1762  of  4551
1763  of  4551
1764  of  4551
1765  of  4551
1766  of  4551
1767  of  4551
1768  of  4551
1769  of  4551
1770  of  4551
1771  of  4551
1772  of  4551
1773  of  4551
1774  of  4551
1775  of  4551
1776  of  4551
1777  of  

2255  of  4551
2256  of  4551
2257  of  4551
2258  of  4551
2259  of  4551
2260  of  4551
2261  of  4551
2262  of  4551
2263  of  4551
2264  of  4551
2265  of  4551
2266  of  4551
2267  of  4551
2268  of  4551
2269  of  4551
2270  of  4551
2271  of  4551
2272  of  4551
2273  of  4551
2274  of  4551
2275  of  4551
2276  of  4551
2277  of  4551
2278  of  4551
2279  of  4551
2280  of  4551
2281  of  4551
2282  of  4551
2283  of  4551
2284  of  4551
2285  of  4551
2286  of  4551
2287  of  4551
2288  of  4551
2289  of  4551
2290  of  4551
2291  of  4551
2292  of  4551
2293  of  4551
2294  of  4551
2295  of  4551
2296  of  4551
2297  of  4551
2298  of  4551
2299  of  4551
2300  of  4551
2301  of  4551
2302  of  4551
2303  of  4551
2304  of  4551
2305  of  4551
2306  of  4551
2307  of  4551
2308  of  4551
2309  of  4551
2310  of  4551
2311  of  4551
2312  of  4551
2313  of  4551
2314  of  4551
2315  of  4551
2316  of  4551
2317  of  4551
2318  of  4551
2319  of  4551
2320  of  4551
2321  of  

2802  of  4551
2803  of  4551
2804  of  4551
2805  of  4551
2806  of  4551
2807  of  4551
2808  of  4551
2809  of  4551
2810  of  4551
2811  of  4551
2812  of  4551
2813  of  4551
2814  of  4551
2815  of  4551
2816  of  4551
2817  of  4551
2818  of  4551
2819  of  4551
2820  of  4551
2821  of  4551
2822  of  4551
2823  of  4551
2824  of  4551
2825  of  4551
2826  of  4551
2827  of  4551
2828  of  4551
2829  of  4551
2830  of  4551
2831  of  4551
2832  of  4551
2833  of  4551
2834  of  4551
2835  of  4551
2836  of  4551
2837  of  4551
2838  of  4551
2839  of  4551
2840  of  4551
2841  of  4551
2842  of  4551
2843  of  4551
2844  of  4551
2845  of  4551
2846  of  4551
2847  of  4551
2848  of  4551
2849  of  4551
2850  of  4551
2851  of  4551
2852  of  4551
2853  of  4551
2854  of  4551
2855  of  4551
2856  of  4551
2857  of  4551
2858  of  4551
2859  of  4551
2860  of  4551
2861  of  4551
2862  of  4551
2863  of  4551
2864  of  4551
2865  of  4551
2866  of  4551
2867  of  4551
2868  of  

3349  of  4551
3350  of  4551
3351  of  4551
3352  of  4551
3353  of  4551
3354  of  4551
3355  of  4551
3356  of  4551
3357  of  4551
3358  of  4551
3359  of  4551
3360  of  4551
3361  of  4551
3362  of  4551
3363  of  4551
3364  of  4551
3365  of  4551
3366  of  4551
3367  of  4551
3368  of  4551
3369  of  4551
3370  of  4551
3371  of  4551
3372  of  4551
3373  of  4551
3374  of  4551
3375  of  4551
3376  of  4551
3377  of  4551
3378  of  4551
3379  of  4551
3380  of  4551
3381  of  4551
3382  of  4551
3383  of  4551
3384  of  4551
3385  of  4551
3386  of  4551
3387  of  4551
3388  of  4551
3389  of  4551
3390  of  4551
3391  of  4551
3392  of  4551
3393  of  4551
3394  of  4551
3395  of  4551
3396  of  4551
3397  of  4551
3398  of  4551
3399  of  4551
3400  of  4551
3401  of  4551
3402  of  4551
3403  of  4551
3404  of  4551
3405  of  4551
3406  of  4551
3407  of  4551
3408  of  4551
3409  of  4551
3410  of  4551
3411  of  4551
3412  of  4551
3413  of  4551
3414  of  4551
3415  of  

3896  of  4551
3897  of  4551
3898  of  4551
3899  of  4551
3900  of  4551
3901  of  4551
3902  of  4551
3903  of  4551
3904  of  4551
3905  of  4551
3906  of  4551
3907  of  4551
3908  of  4551
3909  of  4551
3910  of  4551
3911  of  4551
3912  of  4551
3913  of  4551
3914  of  4551
3915  of  4551
3916  of  4551
3917  of  4551
3918  of  4551
3919  of  4551
3920  of  4551
3921  of  4551
3922  of  4551
3923  of  4551
3924  of  4551
3925  of  4551
3926  of  4551
3927  of  4551
3928  of  4551
3929  of  4551
3930  of  4551
3931  of  4551
3932  of  4551
3933  of  4551
3934  of  4551
3935  of  4551
3936  of  4551
3937  of  4551
3938  of  4551
3939  of  4551
3940  of  4551
3941  of  4551
3942  of  4551
3943  of  4551
3944  of  4551
3945  of  4551
3946  of  4551
3947  of  4551
3948  of  4551
3949  of  4551
3950  of  4551
3951  of  4551
3952  of  4551
3953  of  4551
3954  of  4551
3955  of  4551
3956  of  4551
3957  of  4551
3958  of  4551
3959  of  4551
3960  of  4551
3961  of  4551
3962  of  

4443  of  4551
4444  of  4551
4445  of  4551
4446  of  4551
4447  of  4551
4448  of  4551
4449  of  4551
4450  of  4551
4451  of  4551
4452  of  4551
4453  of  4551
4454  of  4551
4455  of  4551
4456  of  4551
4457  of  4551
4458  of  4551
4459  of  4551
4460  of  4551
4461  of  4551
4462  of  4551
4463  of  4551
4464  of  4551
4465  of  4551
4466  of  4551
4467  of  4551
4468  of  4551
4469  of  4551
4470  of  4551
4471  of  4551
4472  of  4551
4473  of  4551
4474  of  4551
4475  of  4551
4476  of  4551
4477  of  4551
4478  of  4551
4479  of  4551
4480  of  4551
4481  of  4551
4482  of  4551
4483  of  4551
4484  of  4551
4485  of  4551
4486  of  4551
4487  of  4551
4488  of  4551
4489  of  4551
4490  of  4551
4491  of  4551
4492  of  4551
4493  of  4551
4494  of  4551
4495  of  4551
4496  of  4551
4497  of  4551
4498  of  4551
4499  of  4551
4500  of  4551
4501  of  4551
4502  of  4551
4503  of  4551
4504  of  4551
4505  of  4551
4506  of  4551
4507  of  4551
4508  of  4551
4509  of  

In [260]:
with open('data/reviews.json', 'w') as f:
    json.dump(yelp_reviews, f)

In [261]:
error_indices = [264, 591, 592, 1347, 1710, 1711, 1919, 2223, 
                 2254, 2255, 2801, 2802, 3895, 3896, 4442, 4443]

In [262]:
with open('data/reviews.json', 'rb') as f:
    test = json.load(f)

In [270]:
test[4546]

{'phone': '+6469983030',
 'reviews': ["I had Curry chicken katsu ramen and spicy miso along\nSweet &spicy chicken (fried) And everything was perfect! I can't wait to go back and try more of the menu\n\nGood prices and good amount of food to leave you feeling very satisfied .\n\nNice & hoy noodles ( Not soggy) buried under flavorful gravy , corn , chicken , boiled egg ,scallions more vegetables and \xa03 fish cakes wit swirls \xa0 which i enjoyed with the gravy \nI recommend trying one of there teas across from Hot bar \n\nThe vibe of the place is a well lit Zen and Chef Meme has gained my trust that whatever comes out the kitchen at this place is going to be good , refreshing and filling . \n \nAll in All a swell spot for Good Eats that's not too pricey in the Flat Iron district",
  "Took my co workers out for lunch at this spot mid week, the place is nicely decorated, lots of items on the menu, seating is comfortable and the service is average. My rating is based on the Ramen itself, 